In [57]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

# Set up: Create session and People table with the People model.
class People(Base):
    """Characters mentioned and their role in the puzzle."""

    __tablename__ = "people"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False, unique=True)
    role = Column(String, nullable=True)

engine = create_engine("sqlite:///temp_test.db", echo=True)
conn = engine.connect()
session = sessionmaker(bind=engine)()
Base.metadata.create_all(engine)




2024-03-25 07:49:19,517 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-25 07:49:19,518 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("people")
2024-03-25 07:49:19,518 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 07:49:19,519 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("people")
2024-03-25 07:49:19,519 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 07:49:19,520 INFO sqlalchemy.engine.Engine 
CREATE TABLE people (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	role VARCHAR, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2024-03-25 07:49:19,520 INFO sqlalchemy.engine.Engine [no key 0.00030s] ()
2024-03-25 07:49:19,521 INFO sqlalchemy.engine.Engine COMMIT


In [58]:
# Add records to the People table
c1 = People(id=1, name="Sprinkles", role="Test")
c2 = People(id=2, name="Sparkles", role="Test")
session.add(c1)
session.add(c2)
session.commit()

2024-03-25 07:49:19,526 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-25 07:49:19,528 INFO sqlalchemy.engine.Engine INSERT INTO people (id, name, role) VALUES (?, ?, ?)
2024-03-25 07:49:19,529 INFO sqlalchemy.engine.Engine [generated in 0.00103s] [(1, 'Sprinkles', 'Test'), (2, 'Sparkles', 'Test')]
2024-03-25 07:49:19,530 INFO sqlalchemy.engine.Engine COMMIT


In [59]:
# Create some data to query for
data = {"id": 2, "name": "Sparkles", "role": "Test"}

In [60]:
# Generic functionality that uses the incoming data to form sqlalchemy queries 
model = People
filters = []
# Iterate over the dictionary and for each key, and 
# check whether the model has a column of the same name
for k, v in data.items():
    # Get the column from the model or None
    column = getattr(model, k, None)
    # If the model has a matching column and the value is truthy, 
    # then append the column == value statement to the query filters
    # This will add 3 statements to the filters list, which will be joined with AND
    if column and v:
        filters.append((getattr(column, "__eq__")(v)))

In [61]:
# Check the attributes of the filters
[x.__dict__ for x in filters]

[{'_orig': (282860879, 282860834),
  '_propagate_attrs': immutabledict({'compile_state_plugin': 'orm', 'plugin_subject': <Mapper at 0x10dc17e50; People>}),
  'left': Column('id', Integer(), table=<people>, primary_key=True, nullable=False),
  'right': BindParameter('%(4525773344 id)s', 2, type_=Integer()),
  'operator': <function _operator.eq(a, b, /)>,
  'type': Boolean(),
  'negate': <function _operator.ne(a, b, /)>,
  '_is_implicitly_boolean': True,
  'modifiers': {}},
 {'_orig': (282860891, 282860867),
  '_propagate_attrs': immutabledict({'compile_state_plugin': 'orm', 'plugin_subject': <Mapper at 0x10dc17e50; People>}),
  'left': Column('name', String(), table=<people>, nullable=False),
  'right': BindParameter('%(4525773872 name)s', 'Sparkles', type_=String()),
  'operator': <function _operator.eq(a, b, /)>,
  'type': Boolean(),
  'negate': <function _operator.ne(a, b, /)>,
  '_is_implicitly_boolean': True,
  'modifiers': {}},
 {'_orig': (282859508, 282860981),
  '_propagate_attr

In [62]:
# Query using the filters list built by the for loop
result = session.query(model).filter(*filters).one_or_none()

2024-03-25 07:49:19,551 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-25 07:49:19,552 INFO sqlalchemy.engine.Engine SELECT people.id AS people_id, people.name AS people_name, people.role AS people_role 
FROM people 
WHERE people.id = ? AND people.name = ? AND people.role = ?
2024-03-25 07:49:19,553 INFO sqlalchemy.engine.Engine [generated in 0.00053s] (2, 'Sparkles', 'Test')


In [63]:
# Check that it returned the correct record
result.name

'Sparkles'

In [64]:
# Clean up: remove db file
from pathlib import Path

path = Path("temp_test.db")

if path.exists():
    path.unlink()
